In [55]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
from matplotlib.colors import LogNorm, Normalize
import os
import numpy as np

# Define file path and color schemes
file_path = '/Users/yukthabhadane/Documents/Climate Finance Thesis/Paper Alissa Jan 2025/Excel sheets/jupyter To Do 8 and 9 .xlsx'

# Updated color schemes to match the metrics
COLOR_SCHEMES = {
    'Benefits': 'Greens',
    'Avoided Loss, Damages & Adaptation Cost Benefits': 'Purples',
    'Avoided Air Pollution Benefits': 'Oranges',
    'costs': 'BuPu',
    'net_benefits': 'RdBu'  # Removed _r to flip red and blue
}

def process_benefits_data_revised(file_path, data_row):
    """Process benefits data with revised structure handling"""
    df = pd.read_excel(file_path, sheet_name='Benefits', header=None)
    iso_codes = df.iloc[5:, -1].dropna()
    data_values = df.iloc[5:, data_row].dropna()
    
    result_df = pd.DataFrame({
        'Country Code': iso_codes,
        'Value': data_values
    }).dropna()
    
    result_df['Value'] = pd.to_numeric(result_df['Value'], errors='coerce')
    result_df = result_df.dropna()
    
    iso_dict = create_iso_mapping()
    result_df['iso3'] = result_df['Country Code'].map(iso_dict)
    result_df = result_df.dropna(subset=['iso3'])
    
    return result_df[['iso3', 'Value']]

def create_world_map_revised(world_data, data_column, title, metric_type, highlight_countries=None):
    """Create world map with specific color scheme for each metric type"""
    fig, ax = plt.subplots(1, 1, figsize=(20, 8))
    
    # Identify countries with and without data
    world_data['has_data'] = world_data[data_column].notna()
    
    values = world_data[data_column].dropna()
    
    # Special handling for net benefits which can have negative values
    if metric_type == 'net_benefits':
        vmax = max(abs(values.min()), abs(values.max()))
        norm = Normalize(vmin=-vmax, vmax=vmax)
    else:
        vmin = values.min() if len(values) > 0 else 0.0001
        vmax = values.max() if len(values) > 0 else 10
        norm = LogNorm(vmin=max(vmin, 0.0001), vmax=max(vmax, 0.001))
    
    # Use specific color scheme based on metric type
    cmap = COLOR_SCHEMES.get(metric_type, 'viridis')
    
    # Plot all countries with a light border first
    world_data.plot(
        ax=ax,
        color='#f8f8f8',  # Much lighter gray for missing data
        edgecolor='#dddddd',  # Lighter border color
        linewidth=0.2  # Very slight border
    )
    
    # Plot countries with data
    world_data[world_data['has_data']].plot(
        column=data_column,
        ax=ax,
        legend=False,
        missing_kwds={'color': '#f8f8f8'},  # Keep consistent with the base map
        cmap=cmap,
        edgecolor='gray',
        linewidth=0.2,
        norm=norm
    )
    
    if highlight_countries:
        world_data[world_data['ISO_A3'].isin(highlight_countries)].boundary.plot(
            ax=ax, 
            color='black',
            linewidth=1.5
        )
    
    # Add colorbar
    cbar_ax = fig.add_axes([0.4, 0.2, 0.2, 0.01])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm,
                       cax=cbar_ax,
                       orientation='horizontal')
    
    if metric_type == 'net_benefits':
        # Add central line at zero
        cbar_ax.axvline(x=0.5, color='black', linewidth=0.5)
        # Format ticks to show the zero point clearly
        ticks = [-vmax, -vmax/2, 0, vmax/2, vmax]
        cbar.set_ticks(ticks)
        cbar.set_ticklabels([f'{t:,.0f}' for t in ticks])
    
    cbar.ax.tick_params(labelsize=10)
    
    # Add legend for missing data
    ax.add_patch(plt.Rectangle((0.1, 0.2), 0.02, 0.02, 
                              facecolor='#f8f8f8', 
                              edgecolor='#dddddd',
                              linestyle=':',
                              transform=ax.transAxes))
    ax.text(0.13, 0.21, 'No data available', 
            transform=ax.transAxes, 
            fontsize=8)
    
    ax.axis('off')
    plt.title(title, pad=20, fontsize=12)
    return fig

def load_world_map():
    return gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")

def create_iso_mapping():
    return {country.alpha_2: country.alpha_3 
            for country in pycountry.countries 
            if hasattr(country, 'alpha_2') and hasattr(country, 'alpha_3')}

def generate_all_maps():
    """Generate and save all maps"""
    # Create output directory
    output_dir = '/Users/yukthabhadane/Documents/Climate Finance Thesis/Paper Alissa Jan 2025/Maps'
    os.makedirs(output_dir, exist_ok=True)
    
    world = load_world_map()
    highlight_countries = ['IND', 'IDN', 'ZAF', 'MEX', 'VNM', 'IRN', 'THA', 'EGY']
    
    # Define map configurations for Benefits sheet
    benefits_maps_config = [
        # Benefits maps
        {'year': '2025-2030', 'type': 'Benefits', 'scc': '190', 'column': 3, 'sheet': 'Benefits'},
        {'year': '2025-2030', 'type': 'Benefits', 'scc': '1056', 'column': 5, 'sheet': 'Benefits'},
        {'year': '2025-2050', 'type': 'Benefits', 'scc': '190', 'column': 12, 'sheet': 'Benefits'},
        {'year': '2025-2050', 'type': 'Benefits', 'scc': '1056', 'column': 14, 'sheet': 'Benefits'},
        
        # Damages maps
        {'year': '2025-2030', 'type': 'Avoided Loss, Damages & Adaptation Cost Benefits', 'scc': '190', 'column': 7, 'sheet': 'Benefits'},
        {'year': '2025-2030', 'type': 'Avoided Loss, Damages & Adaptation Cost Benefits', 'scc': '1056', 'column': 9, 'sheet': 'Benefits'},
        {'year': '2025-2050', 'type': 'Avoided Loss, Damages & Adaptation Cost Benefits', 'scc': '190', 'column': 16, 'sheet': 'Benefits'},
        {'year': '2025-2050', 'type': 'Avoided Loss, Damages & Adaptation Cost Benefits', 'scc': '1056', 'column': 18, 'sheet': 'Benefits'},
        
        # Air pollution maps
        {'year': '2025-2030', 'type': 'Avoided Air Pollution Benefits', 'scc': None, 'column': 10, 'sheet': 'Benefits'},
        {'year': '2025-2050', 'type': 'Avoided Air Pollution Benefits', 'scc': None, 'column': 19, 'sheet': 'Benefits'}
    ]
    
    # Process maps from Benefits sheet
    for config in benefits_maps_config:
        print(f"Generating {config['type']} map for {config['year']}" + 
              (f" (SCC {config['scc']})" if config['scc'] else ""))
        
        benefits_data = process_benefits_data_revised(file_path, config['column'])
        merged_world = world.merge(benefits_data, how='left', left_on='ISO_A3', right_on='iso3')
        
        title = (f"{config['type']} (Bn$)" + 
                (f"({config['scc']} SCC)" if config['scc'] else "") + 
                f" for {config['year']}")
        
        fig = create_world_map_revised(
            merged_world,
            'Value',
            title,
            config['type'],
            highlight_countries
        )
        
        # Save the figure
        filename = f"{config['type']}_{config['year']}" + (f"_scc{config['scc']}" if config['scc'] else "") + ".png"
        filepath = os.path.join(output_dir, filename.replace(' ', '_'))
        fig.savefig(filepath, bbox_inches='tight', dpi=300)
        plt.close(fig)
    
    # Process Costs and Net Benefits from Decarb sheets
    decarb_configs = [
        {'year': '2025-2030', 'sheet': 'All Cs Decarb 2030', 'country_col': 'Bn $'},
        {'year': '2025-2050', 'sheet': 'All Cs Decarb 2050', 'country_col': 'Country Code'}
    ]
    
    for config in decarb_configs:
        # Read decarb sheet
        decarb_df = pd.read_excel(file_path, sheet_name=config['sheet'])
        
        # Process costs
        print(f"Generating costs map for {config['year']}")
        costs_data = pd.DataFrame({
            'iso3': decarb_df[config['country_col']].map(create_iso_mapping()),
            'Value': decarb_df['Costs ']
        }).dropna()
        
        merged_world = world.merge(costs_data, how='left', left_on='ISO_A3', right_on='iso3')
        fig = create_world_map_revised(
            merged_world,
            'Value',
            f"Costs (Bn$) for {config['year']}",
            'costs',
            highlight_countries
        )
        filepath = os.path.join(output_dir, f"costs_{config['year']}.png")
        fig.savefig(filepath, bbox_inches='tight', dpi=300)
        plt.close(fig)
        
        # Process net benefits for both SCCs
        for scc in ['1056', '190']:
            print(f"Generating net benefits map for {config['year']} (SCC {scc})")
            net_benefits_data = pd.DataFrame({
                'iso3': decarb_df[config['country_col']].map(create_iso_mapping()),
                'Value': decarb_df[f'Net Benefit {scc} SCC']
            }).dropna()
            
            merged_world = world.merge(net_benefits_data, how='left', left_on='ISO_A3', right_on='iso3')
            fig = create_world_map_revised(
                merged_world,
                'Value',
                f"Net Benefits (Bn$)({scc} SCC) for {config['year']}",
                'net_benefits',
                highlight_countries
            )
            filepath = os.path.join(output_dir, f"net_benefits_{config['year']}_scc{scc}.png")
            fig.savefig(filepath, bbox_inches='tight', dpi=300)
            plt.close(fig)
    
    print("\nAll maps have been generated and saved to:", output_dir)

# Run the function to generate all maps
if __name__ == "__main__":
    generate_all_maps()

Generating Benefits map for 2025-2030 (SCC 190)
Generating Benefits map for 2025-2030 (SCC 1056)
Generating Benefits map for 2025-2050 (SCC 190)
Generating Benefits map for 2025-2050 (SCC 1056)
Generating Avoided Loss, Damages & Adaptation Cost Benefits map for 2025-2030 (SCC 190)
Generating Avoided Loss, Damages & Adaptation Cost Benefits map for 2025-2030 (SCC 1056)
Generating Avoided Loss, Damages & Adaptation Cost Benefits map for 2025-2050 (SCC 190)
Generating Avoided Loss, Damages & Adaptation Cost Benefits map for 2025-2050 (SCC 1056)
Generating Avoided Air Pollution Benefits map for 2025-2030
Generating Avoided Air Pollution Benefits map for 2025-2050


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Generating costs map for 2025-2030
Generating net benefits map for 2025-2030 (SCC 1056)
Generating net benefits map for 2025-2030 (SCC 190)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Generating costs map for 2025-2050
Generating net benefits map for 2025-2050 (SCC 1056)
Generating net benefits map for 2025-2050 (SCC 190)

All maps have been generated and saved to: /Users/yukthabhadane/Documents/Climate Finance Thesis/Paper Alissa Jan 2025/Maps
